# Sasicha Chotiprayanakul 6338208121
### Sirintra Kunakornpaiboonsiri 6338237321, Virada Poopipathiranyakul 6338203021 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
customers = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv')
# customers.columns
customers = customers[['customer_id','age','fashion_news_frequency','club_member_status']]
customers.head()


In [ ]:
transactions = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
transactions.columns
# transactions.head()

In [ ]:
transactions['year'] = pd.DatetimeIndex(transactions['t_dat']).year
transactions['month'] = pd.DatetimeIndex(transactions['t_dat']).month
transactions.tail()

In [ ]:
articles = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv')
# articles.columns
articles = articles[['article_id','product_code','product_type_no','graphical_appearance_no','department_no','colour_group_code','section_no','garment_group_no']]
articles.head()

## Find weeks since each transaction

In [ ]:
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])
transactions["week_since"] = (transactions["t_dat"].max() - transactions["t_dat"]).dt.days // 7
transactions["week_since"].value_counts()
transactions.tail()

## Transactions in the past month

In [ ]:
last_month_transactions=transactions[(transactions.week_since>=0) & (transactions.week_since<=4)]
last_month_transactions

## Most popular articles in the past month

In [ ]:
last_month_articles= last_month_transactions["article_id"].value_counts().reset_index().sort_values(by='article_id',ascending=False)[0:24]
last_month_articles.columns= ['article_id','count']
last_month_articles

In [ ]:
freq_trans = last_month_transactions[last_month_transactions['article_id'].isin(last_month_articles['article_id'])]
freq_trans

In [ ]:
freq_customers=customers[customers['customer_id'].isin(freq_trans['customer_id'])]
freq_customers

In [ ]:
freq_customers.loc[freq_customers['fashion_news_frequency']=='NONE','fashion_news_frequency'] = int(0)
freq_customers.loc[freq_customers['fashion_news_frequency']=='Monthly','fashion_news_frequency'] = int(1)
freq_customers.loc[freq_customers['fashion_news_frequency']=='Regularly','fashion_news_frequency'] = int(2)

freq_customers.loc[freq_customers['club_member_status']=='LEFT CLUB','club_member_status'] = int(0)
freq_customers.loc[freq_customers['club_member_status']=='PRE-CREATE','club_member_status'] = int(1)
freq_customers.loc[freq_customers['club_member_status']=='ACTIVE','club_member_status'] = int(2)

freq_customers

In [ ]:
not_freq_customers=customers[~customers['customer_id'].isin(freq_trans['customer_id'])]
not_freq_customers

In [ ]:
freq_info = freq_trans.merge(freq_customers,on='customer_id').merge(last_month_articles,on='article_id')
freq_info

In [ ]:
freq_info=freq_info.dropna()
freq_info=freq_info.drop_duplicates()
freq_info

In [ ]:
X = freq_info[['article_id','price','sales_channel_id','week_since','age','fashion_news_frequency','club_member_status']]
y = freq_info['article_id']

In [ ]:
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

In [ ]:
print('Shape of X training set {}'.format(X_trainset.shape),'&',' Size of Y training set {}'.format(y_trainset.shape))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
articleTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
articleTree

In [ ]:
articleTree.fit(X_trainset,y_trainset)

In [ ]:
predTree = articleTree.predict(X_testset)

In [ ]:
print (predTree [0:5])
print (y_testset [0:5])

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Train set Accuracy: ", metrics.accuracy_score(y_trainset, articleTree.predict(X_trainset)))
print("DecisionTrees's Test set Accuracy: ", metrics.accuracy_score(y_testset, predTree))

In [ ]:
pip install pydotplus

In [ ]:
from  io import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = "articletree.png"
featureNames = freq_info.columns[0:7]
out=tree.export_graphviz(articleTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')